In [1]:
headers = ({'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36'})

In [2]:
import requests
from bs4 import BeautifulSoup

base_url = "https://www.arabam.com/ikinci-el/otomobil/toyota-corolla?take=50&page="
car_data = []

for page_number in range(1, 51):
    url = base_url + str(page_number)
    data = requests.get(url).text
    soup = BeautifulSoup(data, 'html.parser')
    listings = soup.find_all('tr', class_='listing-list-item should-hover bg-white')
    for listing in listings:
        model = listing.find('div', class_='listing-text-new').text
        year = listing.find('td', class_='listing-text').text.strip()
        kilometers = listing.find_all('td', class_='listing-text')[1].text.strip()
        color = listing.find_all('td', class_='listing-text')[2].text.strip()
        price = listing.find('span', class_='db no-wrap listing-price').text.strip()
            
        location = listing.find('span', class_='').text + " - " + listing.find_all('span', class_='')[1].text

        car_data.append([model, year, kilometers, color, price, location])

In [3]:
import pandas as pd
df = pd.DataFrame(car_data, columns=['Model', 'Year', 'Kilometers', 'Color', 'Price', 'Location'])
df

,Model,Year,Kilometers,Color,Price,Location
0,Toyota Corolla 1.4 D-4D Advance,2014,245.000,Beyaz,700.000 TL,Tekirdağ - Çerkezköy
1,Toyota Corolla 1.6 Terra Special,1999,265.000,Yeşil,364.000 TL,Sivas - Merkez
2,Toyota Corolla 1.6 Terra,2006,146.000,Siyah,670.000 TL,İstanbul - Çekmeköy
3,Toyota Corolla 1.8 Hybrid Dream,2021,62.600,Gri,994.000 TL,İstanbul - Şişli
4,Toyota Corolla 1.3 ECO,2000,370.000,Beyaz,293.000 TL,Manisa - Akhisar
...,...,...,...,...,...,...
2484,Toyota Corolla 1.3 XL,1993,297.000,Gri (metalik),270.000 TL,Bilecik - Bozüyük
2485,Toyota Corolla 1.33 Life,2015,70.200,Beyaz,710.000 TL,İstanbul - Büyükçekmece
2486,Toyota Corolla 1.4 D-4D Active,2014,132.000,Beyaz,755.000 TL,Siirt - Merkez
2487,Toyota Corolla 1.6 Terra,2003,320.000,Bordo,355.500 TL,Giresun - Merkez


In [4]:
col = df["Model"].str.split(" ", 1, expand=True).rename({0:"Car Brand", 1:"CarModel"}, axis = 1)

In [5]:
df = df.join([col])

In [6]:
df = df.drop("Model", axis = 1)

In [7]:
df.isnull().sum()

Year          0
Kilometers    0
Color         0
Price         0
Location      0
Car Brand     0
CarModel      0
dtype: int64

In [8]:
col2 = df.CarModel.str.split(" ", 1, expand=True).rename({0:"Model", 1:"Engine"}, axis = 1)

In [9]:
df = df.join(col2)

In [10]:
df = df.drop('CarModel', axis = 1)

In [11]:
col3 = df.Price.str.split(" ", 1, expand = True).rename({0:"Price1", 1:"Currency"}, axis = 1)

In [12]:
df = df.join(col3)
df

,Year,Kilometers,Color,Price,Location,Car Brand,Model,Engine,Price1,Currency
0,2014,245.000,Beyaz,700.000 TL,Tekirdağ - Çerkezköy,Toyota,Corolla,1.4 D-4D Advance,700.000,TL
1,1999,265.000,Yeşil,364.000 TL,Sivas - Merkez,Toyota,Corolla,1.6 Terra Special,364.000,TL
2,2006,146.000,Siyah,670.000 TL,İstanbul - Çekmeköy,Toyota,Corolla,1.6 Terra,670.000,TL
3,2021,62.600,Gri,994.000 TL,İstanbul - Şişli,Toyota,Corolla,1.8 Hybrid Dream,994.000,TL
4,2000,370.000,Beyaz,293.000 TL,Manisa - Akhisar,Toyota,Corolla,1.3 ECO,293.000,TL
...,...,...,...,...,...,...,...,...,...,...
2484,1993,297.000,Gri (metalik),270.000 TL,Bilecik - Bozüyük,Toyota,Corolla,1.3 XL,270.000,TL
2485,2015,70.200,Beyaz,710.000 TL,İstanbul - Büyükçekmece,Toyota,Corolla,1.33 Life,710.000,TL
2486,2014,132.000,Beyaz,755.000 TL,Siirt - Merkez,Toyota,Corolla,1.4 D-4D Active,755.000,TL
2487,2003,320.000,Bordo,355.500 TL,Giresun - Merkez,Toyota,Corolla,1.6 Terra,355.500,TL


In [13]:
if "Price" in df.columns:
    df = df.drop("Price", axis=1)
else:
    print("Column 'Price' not found in the dataset.")

print(df.head())

   Year Kilometers  Color              Location Car Brand    Model  \
0  2014    245.000  Beyaz  Tekirdağ - Çerkezköy    Toyota  Corolla   
1  1999    265.000  Yeşil        Sivas - Merkez    Toyota  Corolla   
2  2006    146.000  Siyah   İstanbul - Çekmeköy    Toyota  Corolla   
3  2021     62.600    Gri      İstanbul - Şişli    Toyota  Corolla   
4  2000    370.000  Beyaz      Manisa - Akhisar    Toyota  Corolla   

              Engine   Price1 Currency  
0   1.4 D-4D Advance  700.000       TL  
1  1.6 Terra Special  364.000       TL  
2          1.6 Terra  670.000       TL  
3   1.8 Hybrid Dream  994.000       TL  
4            1.3 ECO  293.000       TL  


In [14]:
col4 = df.Location.str.split(" ", 1, expand = True).rename({0:"City", 1:"District"}, axis = 1)
col4

,City,District
0,Tekirdağ,- Çerkezköy
1,Sivas,- Merkez
2,İstanbul,- Çekmeköy
3,İstanbul,- Şişli
4,Manisa,- Akhisar
...,...,...
2484,Bilecik,- Bozüyük
2485,İstanbul,- Büyükçekmece
2486,Siirt,- Merkez
2487,Giresun,- Merkez


In [15]:
df = df.join(col4)

In [16]:
df = df.drop("Location", axis = 1)

In [17]:
df['District'] = df['District'].str.replace("-", " ")

In [18]:
df.rename(columns = {"Price1":"Price"}, inplace = True)

In [19]:
df['Year'] = df['Year'].astype(int)

In [20]:
df['Kilometers'] = df['Kilometers'].str.replace('.', '')

C:\Users\user\AppData\Local\Temp\ipykernel_12896\2414687831.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Kilometers'] = df['Kilometers'].str.replace('.', '')


In [21]:
df['Kilometers'] = df['Kilometers'].str.replace('-', '').astype(int)

In [22]:
df['Kilometers'].astype(int)

0       245000
1       265000
2       146000
3        62600
4       370000
         ...  
2484    297000
2485     70200
2486    132000
2487    320000
2488     73000
Name: Kilometers, Length: 2489, dtype: int32

In [23]:
type(df['Kilometers'][0])

numpy.int32

In [24]:
df = df.drop("Currency", axis = 1)

In [25]:
col6 = df['Engine'].str.split(' ', n=1, expand=True).rename({0: 'Engine1', 1: 'Devami'}, axis=1)
col6

,Engine1,Devami
0,1.4,D-4D Advance
1,1.6,Terra Special
2,1.6,Terra
3,1.8,Hybrid Dream
4,1.3,ECO
...,...,...
2484,1.3,XL
2485,1.33,Life
2486,1.4,D-4D Active
2487,1.6,Terra


In [26]:
df = df.join(col6)

In [27]:
df['Engine1'] = df['Engine1'].astype(float)

In [28]:
df = df.drop(columns = 'Engine')

In [29]:
df.rename(columns = {"Engine1":"Engine"}, inplace = True)

In [30]:
df = df.drop(columns = 'Devami')

In [31]:
df['Price'] = df['Price'].str.replace('.', '').astype(int)

C:\Users\user\AppData\Local\Temp\ipykernel_12896\3128332942.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Price'] = df['Price'].str.replace('.', '').astype(int)


In [32]:
df.reset_index(drop = True, inplace = True)
df

,Year,Kilometers,Color,Car Brand,Model,Price,City,District,Engine
0,2014,245000,Beyaz,Toyota,Corolla,700000,Tekirdağ,Çerkezköy,1.40
1,1999,265000,Yeşil,Toyota,Corolla,364000,Sivas,Merkez,1.60
2,2006,146000,Siyah,Toyota,Corolla,670000,İstanbul,Çekmeköy,1.60
3,2021,62600,Gri,Toyota,Corolla,994000,İstanbul,Şişli,1.80
4,2000,370000,Beyaz,Toyota,Corolla,293000,Manisa,Akhisar,1.30
...,...,...,...,...,...,...,...,...,...
2484,1993,297000,Gri (metalik),Toyota,Corolla,270000,Bilecik,Bozüyük,1.30
2485,2015,70200,Beyaz,Toyota,Corolla,710000,İstanbul,Büyükçekmece,1.33
2486,2014,132000,Beyaz,Toyota,Corolla,755000,Siirt,Merkez,1.40
2487,2003,320000,Bordo,Toyota,Corolla,355500,Giresun,Merkez,1.60


In [33]:
df.to_csv('car-sale-corolla.csv', index = False)